# Text Generation

In [1]:
# ! pip install evaluate
# !pip install transformers datasets tokenizers seqeval -q

In [2]:
# Import libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import datasets
# from datasets import Dataset
# from datasets import load_dataset
import evaluate

import transformers
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline

from sklearn.model_selection import train_test_split
import os

In [3]:
df_nlp = pd.read_json('../data/banking_responses_1000.jsonl', lines = True)

In [4]:
df_nlp.head()

,key,transcript,convenient,speed,informative
0,1,I always receive quick transfer on transaction...,easy booking,quick transfer,detailed fees
1,2,The loan officer was very helpful and card rep...,simple setup,fast service,card replacement
2,3,The loan officer was very helpful and spending...,online update,quick transfer,spending insights
3,4,"I managed to simple setup on their website, sa...",simple setup,five minutes,spending insights
4,5,Setting up my account online was smooth naviga...,smooth navigation,one minute,complex queries


# Transform Data

In [5]:
df_nlp['tokens'] = df_nlp['transcript'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['convenient'] = df_nlp['convenient'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['speed'] = df_nlp['speed'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['informative'] = df_nlp['informative'].apply(lambda x: re.findall(r'[.?]|\w+', x))

In [6]:
def map_tags(lst, input:str):
  dct = {}
  print(lst)
  begin = ""
  inbet = ""
  if input == "convenient":
    begin = 1
    inbet = 2
  elif input == "speed":
    begin = 3
    inbet = 4
  elif input == "informative":
    begin = 5
    inbet = 6
  dct[lst[0]] = begin
  for i in range(1, len(lst)):
    dct[lst[i]] = inbet
  # print(dct)
  return dct

In [7]:
df_nlp['convenient'] = df_nlp['convenient'].apply(lambda x: map_tags(x, 'convenient'))
df_nlp['speed'] = df_nlp['speed'].apply(lambda x: map_tags(x, 'speed'))
df_nlp['informative'] = df_nlp['informative'].apply(lambda x: map_tags(x, 'informative'))

['easy', 'booking']
['simple', 'setup']
['online', 'update']
['simple', 'setup']
['smooth', 'navigation']
['online', 'update']
['easy', 'to', 'use']
['simple', 'setup']
['online', 'update']
['intuitive', 'interface']
['simple', 'setup']
['simple', 'setup']
['simple', 'setup']
['smooth', 'navigation']
['online', 'update']
['simple', 'setup']
['easy', 'booking']
['smooth', 'navigation']
['easy', 'to', 'use']
['intuitive', 'interface']
['simple', 'setup']
['online', 'update']
['intuitive', 'interface']
['intuitive', 'interface']
['simple', 'setup']
['simple', 'setup']
['simple', 'setup']
['intuitive', 'interface']
['online', 'update']
['easy', 'booking']
['simple', 'setup']
['simple', 'setup']
['smooth', 'navigation']
['intuitive', 'interface']
['easy', 'to', 'use']
['easy', 'to', 'use']
['easy', 'booking']
['easy', 'booking']
['smooth', 'navigation']
['smooth', 'navigation']
['online', 'update']
['simple', 'setup']
['intuitive', 'interface']
['easy', 'to', 'use']
['easy', 'booking']
['on

In [8]:
df_nlp.head()

,key,transcript,convenient,speed,informative,tokens
0,1,I always receive quick transfer on transaction...,"{'easy': 1, 'booking': 2}","{'quick': 3, 'transfer': 4}","{'detailed': 5, 'fees': 6}","[I, always, receive, quick, transfer, on, tran..."
1,2,The loan officer was very helpful and card rep...,"{'simple': 1, 'setup': 2}","{'fast': 3, 'service': 4}","{'card': 5, 'replacement': 6}","[The, loan, officer, was, very, helpful, and, ..."
2,3,The loan officer was very helpful and spending...,"{'online': 1, 'update': 2}","{'quick': 3, 'transfer': 4}","{'spending': 5, 'insights': 6}","[The, loan, officer, was, very, helpful, and, ..."
3,4,"I managed to simple setup on their website, sa...","{'simple': 1, 'setup': 2}","{'five': 3, 'minutes': 4}","{'spending': 5, 'insights': 6}","[I, managed, to, simple, setup, on, their, web..."
4,5,Setting up my account online was smooth naviga...,"{'smooth': 1, 'navigation': 2}","{'one': 3, 'minute': 4}","{'complex': 5, 'queries': 6}","[Setting, up, my, account, online, was, smooth..."


In [9]:
df = df_nlp[['key', 'tokens', 'convenient', 'speed', 'informative']]
df = df.rename(columns={'key':'id'})

In [10]:
def map_num(row):
	curr_dict = row['convenient'].copy()
	curr_dict.update(row['speed'])
	curr_dict.update(row['informative'])
	return [0 if curr_dict.get(i) is None else curr_dict.get(i) for i in row['tokens']]


In [11]:
df['ner_tags'] = df.apply(map_num, axis=1)
df.head()

,id,tokens,convenient,speed,informative,ner_tags
0,1,"[I, always, receive, quick, transfer, on, tran...","{'easy': 1, 'booking': 2}","{'quick': 3, 'transfer': 4}","{'detailed': 5, 'fees': 6}","[0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,"[The, loan, officer, was, very, helpful, and, ...","{'simple': 1, 'setup': 2}","{'fast': 3, 'service': 4}","{'card': 5, 'replacement': 6}","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, ..."
2,3,"[The, loan, officer, was, very, helpful, and, ...","{'online': 1, 'update': 2}","{'quick': 3, 'transfer': 4}","{'spending': 5, 'insights': 6}","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, ..."
3,4,"[I, managed, to, simple, setup, on, their, web...","{'simple': 1, 'setup': 2}","{'five': 3, 'minutes': 4}","{'spending': 5, 'insights': 6}","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,5,"[Setting, up, my, account, online, was, smooth...","{'smooth': 1, 'navigation': 2}","{'one': 3, 'minute': 4}","{'complex': 5, 'queries': 6}","[0, 0, 0, 0, 0, 0, 1, 2, 0, 5, 6, 0, 0, 0, 0, ..."


In [12]:
label_list = ['O', 'B-CON', "I-CON", "B-SPD", "I-SPD", "B-INF", "I-INF"]
random_seed = 1
transformers.set_seed(seed=random_seed)

In [13]:
train_val, test = train_test_split(df, test_size=0.2, random_state=random_seed)

train_val = datasets.Dataset.from_pandas(train_val)
test = datasets.Dataset.from_pandas(test)

train_val = train_val.train_test_split(test_size=0.1, shuffle=True)
train, val = train_val['train'], train_val['test']

_nlp_data = {
	'train' : train,
	'validation' : val,
	'test' : test
}

nlp_data = datasets.DatasetDict(_nlp_data)

# Load in the dataset

# Tokenization

In [14]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

c:\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
tokenized_datasets = nlp_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

# Model Training

In [17]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [18]:
import numpy as np
import evaluate

# Assuming label_list is defined elsewhere in your code
metric = evaluate.load("seqeval")  # Load the seqeval metric

def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    # Get the predicted class labels by taking argmax
    pred_logits = np.argmax(pred_logits, axis=2)

    # Remove all values where the label is -100 and create predictions
    predictions = [
        [label_list[eval_pred] for (eval_pred, l) in zip(prediction, label) if l != -100 and 0 <= eval_pred < len(label_list)]
        for prediction, label in zip(pred_logits, labels)
    ]

    # Remove all values where the label is -100 and create true labels
    true_labels = [
        [label_list[l] for (l, eval_pred) in zip(label, prediction) if l != -100 and 0 <= l < len(label_list)]
        for label, prediction in zip(labels, pred_logits)
    ]

    if len(predictions)!= len(true_labels):
      print(len(predictions), len(true_labels))
      print("not equal len!")

    # Calculate metrics
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [19]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

args = TrainingArguments(
	"test-ner",
	evaluation_strategy = "epoch",
	learning_rate=2e-5,
	per_device_train_batch_size=16,
	per_device_eval_batch_size=16,
	num_train_epochs=30,
	weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

# I changed this part to the evaluate function
metric = evaluate.load("seqeval")

trainer = Trainer(
    model,
	args,
	train_dataset=tokenized_datasets["train"],
	eval_dataset=tokenized_datasets["validation"],
	data_collator=data_collator,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


$$ F1-score = \frac{1}{Precision^{-1} + Recall^{-1}} $$

In [20]:
trainer.train()

  0%|          | 0/1350 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.10234147310256958, 'eval_precision': 1.0, 'eval_recall': 0.9154929577464789, 'eval_f1': 0.9558823529411764, 'eval_accuracy': 0.9902234636871509, 'eval_runtime': 5.5171, 'eval_samples_per_second': 14.5, 'eval_steps_per_second': 0.906, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.02153688296675682, 'eval_precision': 1.0, 'eval_recall': 0.9507042253521126, 'eval_f1': 0.9747292418772564, 'eval_accuracy': 0.9937150837988827, 'eval_runtime': 4.086, 'eval_samples_per_second': 19.579, 'eval_steps_per_second': 1.224, 'epoch': 2.0}


...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.012054452672600746, 'eval_precision': 0.9927536231884058, 'eval_recall': 0.9647887323943662, 'eval_f1': 0.9785714285714285, 'eval_accuracy': 0.994413407821229, 'eval_runtime': 4.425, 'eval_samples_per_second': 18.079, 'eval_steps_per_second': 1.13, 'epoch': 29.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.012123851105570793, 'eval_precision': 0.9927536231884058, 'eval_recall': 0.9647887323943662, 'eval_f1': 0.9785714285714285, 'eval_accuracy': 0.994413407821229, 'eval_runtime': 4.6161, 'eval_samples_per_second': 17.33, 'eval_steps_per_second': 1.083, 'epoch': 30.0}
{'train_runtime': 4711.0135, 'train_samples_per_second': 4.585, 'train_steps_per_second': 0.287, 'train_loss': 0.03620204007184064, 'epoch': 30.0}


TrainOutput(global_step=1350, training_loss=0.03620204007184064, metrics={'train_runtime': 4711.0135, 'train_samples_per_second': 4.585, 'train_steps_per_second': 0.287, 'total_flos': 266028454595904.0, 'train_loss': 0.03620204007184064, 'epoch': 30.0})

# Save model

In [21]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [22]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id

# Model Prediction

In [23]:
json.dump(config, open("ner_model/config.json","w"))

In [24]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [25]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "The ATM at Bank XYZ was super convenient and had English instructions."

ner_results = nlp(example)

for i in ner_results:
	print(i)

{'entity': 'I-INF', 'score': 0.9641136, 'index': 14, 'word': 'instructions', 'start': 57, 'end': 69}


In [26]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "I appreciated how quickly my transaction was processed by the teller."

ner_results = nlp(example)

for i in ner_results:
	print(i)